In [1]:
import numpy as np # 数据处理最重要的模块
import pandas as pd # 数据处理最重要的模块
import matplotlib.pyplot as plt  # 画图模块
import scipy.stats as stats # 统计模块
import scipy
from datetime import datetime # 时间模块
from IPython.core.interactiveshell import InteractiveShell # jupyter运行输出的模块
import statsmodels.formula.api as smf  # OLS regression

#输出矢量图 渲染矢量图 是一个魔法函数（Magic Functions）内嵌绘图
%matplotlib inline 
%config InlineBackend.figure_format = 'svg'

#显示每一个运行结果
InteractiveShell.ast_node_interactivity = 'all'

#设置行不限制数量
#pd.set_option('display.max_rows',None)

#设置列不限制数量
pd.set_option('display.max_columns', None)

In [4]:
data = pd.read_csv('datasets/1990.12.19-今股票上证指数数据.csv')
data['Day'] = pd.to_datetime(data['Day'],format='%Y/%m/%d')
data.set_index('Day', inplace = True)
data.sort_values(by = ['Day'],axis=0, ascending=True)

,Preclose,Open,Highest,Lowest,Close,Volume,Money
Day,,,,,,,
1990-12-19,,96.05,99.98,95.79,99.98,126000,4.940000e+05
1990-12-20,99.98,104.30,104.39,99.98,104.39,19700,8.400000e+04
1990-12-21,104.39,109.07,109.13,103.73,109.13,2800,1.600000e+04
1990-12-24,109.13,113.57,114.55,109.13,114.55,3200,3.100000e+04
1990-12-25,114.55,120.09,120.25,114.55,120.25,1500,6.000000e+03
...,...,...,...,...,...,...,...
2022-07-25,3269.97,3269.71,3273.18,3243.03,3250.39,27124574400,3.480000e+11
2022-07-26,3250.39,3254.19,3282.41,3246.04,3277.44,25946867600,3.340000e+11
2022-07-27,3277.44,3271.78,3282.57,3265.73,3275.76,24913148500,3.400000e+11


In [5]:
daily_data = data['1995-01':'2022-07'].copy()
daily_data['Close'] = pd.to_numeric(daily_data['Close'])
daily_data['Preclose'] = pd.to_numeric(daily_data['Preclose'])
# 计算000001上证指数日收益率 两种：
daily_data['Raw_return'] = daily_data['Close'] / daily_data['Preclose'] - 1
daily_data['Log_return'] = np.log(daily_data['Close']) - np.log(daily_data['Preclose'])
daily_data

,Preclose,Open,Highest,Lowest,Close,Volume,Money,Raw_return,Log_return
Day,,,,,,,,,
1995-01-03,647.87,637.72,647.71,630.53,639.88,23451800,1.806930e+08,-0.012333,-0.012409
1995-01-04,639.88,641.90,655.51,638.86,653.81,42222000,3.069230e+08,0.021770,0.021536
1995-01-05,653.81,655.38,657.52,645.81,646.89,43012300,3.015330e+08,-0.010584,-0.010641
1995-01-06,646.89,642.75,643.89,636.33,640.76,48748200,3.537580e+08,-0.009476,-0.009521
1995-01-09,640.76,637.52,637.55,625.04,626.00,50985100,3.985190e+08,-0.023035,-0.023305
...,...,...,...,...,...,...,...,...,...
2022-07-25,3269.97,3269.71,3273.18,3243.03,3250.39,27124574400,3.480000e+11,-0.005988,-0.006006
2022-07-26,3250.39,3254.19,3282.41,3246.04,3277.44,25946867600,3.340000e+11,0.008322,0.008288
2022-07-27,3277.44,3271.78,3282.57,3265.73,3275.76,24913148500,3.400000e+11,-0.000513,-0.000513


In [6]:
Month_data = daily_data.resample('m')['Log_return'].sum().to_frame()
Month_data['Raw_return'] = np.exp(Month_data['Log_return']) - 1
Month_data.reset_index(inplace=True)
Month_data.rename(columns={'Day':'month'},inplace=True)
Month_data.set_index('month',inplace=True)
Month_data

,Log_return,Raw_return
month,,
1995-01-31,-0.141139,-0.131631
1995-02-28,-0.023979,-0.023694
1995-03-31,0.163651,0.177803
1995-04-30,-0.109315,-0.103552
1995-05-31,0.188901,0.207922
...,...,...
2022-03-31,-0.062604,-0.060685
2022-04-30,-0.065154,-0.063077
2022-05-31,0.044724,0.045739


In [7]:
Quarter_data = daily_data.resample('Q')['Log_return'].sum().to_frame()
Quarter_data['Raw_return'] = np.exp(Quarter_data['Log_return']) - 1
Quarter_data

,Log_return,Raw_return
Day,,
1995-03-31,-0.001467,-0.001466
1995-06-30,-0.025583,-0.025258
1995-09-30,0.135980,0.145660
1995-12-31,-0.263130,-0.231358
1996-03-31,0.001979,0.001981
...,...,...
2021-09-30,-0.006434,-0.006413
2021-12-31,0.019870,0.020069
2022-03-31,-0.112592,-0.106484


In [8]:
Year_data = daily_data.resample('Y')['Log_return'].sum().to_frame()
Year_data['Raw_return'] = np.exp(Year_data['Log_return']) - 1
Year_data

,Log_return,Raw_return
Day,,
1995-12-31,-0.154200,-0.142899
1996-12-31,0.501639,0.651425
1997-12-31,0.264019,0.302153
1998-12-31,-0.040505,-0.039695
1999-12-31,0.175423,0.191750
2000-12-31,0.416917,0.517277
2001-12-31,-0.230898,-0.206180
2002-12-31,-0.192575,-0.175167
2003-12-31,0.097735,0.102670


In [9]:
inflation = pd.read_csv('datasets/inflation.csv')
inflation['month'] = pd.to_datetime(inflation['month'],format='%Y/%m/%d')
inflation.set_index('month',inplace=True)
inflation.sort_values(by=['month'],axis=0,ascending=True)

,cpi
month,
1987-01-31,5.1
1987-02-28,5.4
1987-03-31,5.8
1987-04-30,6.7
1987-05-31,7.6
...,...
2023-03-31,0.7
2023-04-30,0.1
2023-05-31,0.2


In [10]:
market_variance = daily_data.resample('M').apply({
    'Raw_return':
    lambda x: sum(x**2)
})
market_variance.reset_index(inplace=True)
market_variance.rename(columns={'Day':'month','Raw_return':'MV'},inplace=True)
market_variance.set_index('month',inplace=True)
market_variance

,MV
month,
1995-01-31,0.005695
1995-02-28,0.018086
1995-03-31,0.013378
1995-04-30,0.008281
1995-05-31,0.148387
...,...
2022-03-31,0.006740
2022-04-30,0.006234
2022-05-31,0.002035


In [11]:
cross = pd.read_csv('datasets/cross_section.csv')
from pandas.tseries.offsets import MonthEnd
cross['month'] = pd.to_datetime(cross['month'], format='%b %Y') + MonthEnd(1)
cross['to_v'] = pd.to_numeric(cross['to_v'])
cross['floatingvalue'] = pd.to_numeric(cross['floatingvalue'])
# # drop column 1
cross.drop(columns=['Unnamed: 0'], inplace=True)
cross = cross.dropna(subset=['to_v','floatingvalue'])
# cross = cross.dropna()
cross

,Stkcd,month,Rank,Freq,floatingvalue,totalvalue,sizef,sizet,Return,rfmonth,ret,next_ret,next_ret2,next_ret3,next_ret4,next_ret5,next_ret6,next_ret7,next_ret8,next_ret9,next_ret10,next_ret11,next_ret12,beta_6m,N6m,beta_12m,N12m,beta_1y,N1y,beta_5y,N5y,to_v,to_m
0,1,1991-04-30,1,20.0,1.157520e+09,2.118487e+09,20.869546,21.473968,NaN,0.006651,NaN,-0.128345,-0.119551,-0.137013,-0.417680,-0.039425,0.849080,0.016213,0.061181,0.055237,0.006749,0.239957,0.471835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000506,0.000506
1,1,1991-05-31,2,24.0,1.016010e+09,1.859497e+09,20.739149,21.343572,-0.122253,0.006092,-0.128345,-0.119551,-0.137013,-0.417680,-0.039425,0.849080,0.016213,0.061181,0.055237,0.006749,0.239957,0.471835,0.167934,4.192309,44.0,4.192309,44.0,NaN,1.0,NaN,1.0,0.007087,0.007085
2,1,1991-06-30,3,23.0,9.007350e+08,1.648521e+09,20.618722,21.223144,-0.113459,0.006092,-0.119551,-0.137013,-0.417680,-0.039425,0.849080,0.016213,0.061181,0.055237,0.006749,0.239957,0.471835,0.167934,-0.076888,0.246808,67.0,0.246808,67.0,-1.821836,2.0,-1.821836,2.0,0.001155,0.001154
3,1,1991-07-31,4,16.0,7.828100e+08,1.432695e+09,20.478401,21.082823,-0.130921,0.006092,-0.137013,-0.417680,-0.039425,0.849080,0.016213,0.061181,0.055237,0.006749,0.239957,0.471835,0.167934,-0.076888,0.079622,0.165424,83.0,0.165424,83.0,0.807037,3.0,0.807037,3.0,0.000230,0.000231
4,1,1991-08-31,5,15.0,6.748338e+08,1.346275e+09,20.329977,21.020607,-0.411588,0.006092,-0.417680,-0.039425,0.849080,0.016213,0.061181,0.055237,0.006749,0.239957,0.471835,0.167934,-0.076888,0.079622,-0.101487,1.506699,98.0,1.506699,98.0,23.378197,4.0,23.378197,4.0,0.072087,0.071757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695742,605599,2022-08-31,12,23.0,7.933336e+08,7.933334e+09,20.491754,22.794339,-0.059908,0.001241,-0.061149,-0.090457,-0.028152,0.049644,0.070338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.895807,126.0,0.769963,235.0,1.241669,11.0,1.241669,11.0,0.565746,0.565471
695743,605599,2022-09-30,13,21.0,5.450237e+09,7.225556e+09,22.418925,22.700890,-0.089216,0.001241,-0.090457,-0.028152,0.049644,0.070338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.863212,124.0,0.728424,243.0,1.248894,12.0,1.248894,12.0,0.120495,0.120449
695744,605599,2022-10-31,14,16.0,5.303567e+09,7.031111e+09,22.391646,22.673611,-0.026911,0.001241,-0.028152,0.049644,0.070338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.535141,121.0,0.689392,243.0,1.254507,12.0,1.225688,13.0,0.046909,0.046907
695745,605599,2022-11-30,15,22.0,5.573439e+09,7.388889e+09,22.441278,22.723243,0.050885,0.001241,0.049644,0.070338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.467310,124.0,0.685020,243.0,1.083832,12.0,1.103585,14.0,0.068665,0.068614


In [12]:
turnover = pd.DataFrame(cross.groupby(['month']).apply(
    lambda x:
    np.average(x['to_v'],weights=x['floatingvalue']) 
))
turnover = turnover.rename(columns={0:'to'})
turnover

,to
month,
1990-12-31,0.001785
1991-01-31,0.019217
1991-02-28,0.038486
1991-03-31,0.014105
1991-04-30,0.014399
...,...
2022-08-31,0.312049
2022-09-30,0.216061
2022-10-31,0.194497


In [13]:
price = pd.read_csv('datasets/priceratio.csv')
price['month'] = pd.date_range(start='2000',end='2022',freq='M')
price.set_index('month',inplace=True)
price

,pd,pb,pe
month,,,
2000-01-31,5.311050,1.445262,3.695553
2000-02-29,5.429869,1.553916,3.788974
2000-03-31,5.493947,1.590367,3.820005
2000-04-30,5.474707,1.605969,3.888418
2000-05-31,5.453072,1.633036,3.915735
...,...,...,...
2021-08-31,4.082353,0.515687,2.761659
2021-09-30,4.078039,0.520634,2.767360
2021-10-31,4.068158,0.497318,2.750286


In [14]:
reg_data = pd.merge(Month_data,market_variance,on = 'month')
reg_data = pd.merge(reg_data,inflation,on = 'month')
reg_data = pd.merge(reg_data,price,on='month')
reg_data = pd.merge(reg_data,turnover,on='month')
reg_data

,Log_return,Raw_return,MV,cpi,pd,pb,pe,to
month,,,,,,,,
2000-01-31,0.116219,0.123242,0.008258,-0.2,5.311050,1.445262,3.695553,0.481117
2000-02-29,0.110638,0.116990,0.013547,0.7,5.429869,1.553916,3.788974,0.605590
2000-03-31,0.048741,0.049948,0.007877,-0.2,5.493947,1.590367,3.820005,0.773570
2000-04-30,0.019855,0.020053,0.002494,-0.3,5.474707,1.605969,3.888418,0.482107
2000-05-31,0.031218,0.031710,0.003515,0.1,5.453072,1.633036,3.915735,0.355027
...,...,...,...,...,...,...,...,...
2021-08-31,0.042240,0.043145,0.001956,0.8,4.082353,0.515687,2.761659,0.421353
2021-09-30,0.006814,0.006837,0.001526,0.7,4.078039,0.520634,2.767360,0.366907
2021-10-31,-0.005855,-0.005838,0.000690,1.5,4.068158,0.497318,2.750286,0.226861


In [15]:
reg_data['lto'] = reg_data['to'].shift(1)
reg_data['lcpi'] = reg_data['cpi'].shift(2)
reg_data['lpd'] = reg_data['pd'].shift(1)
reg_data

,Log_return,Raw_return,MV,cpi,pd,pb,pe,to,lto,lcpi,lpd
month,,,,,,,,,,,
2000-01-31,0.116219,0.123242,0.008258,-0.2,5.311050,1.445262,3.695553,0.481117,NaN,NaN,NaN
2000-02-29,0.110638,0.116990,0.013547,0.7,5.429869,1.553916,3.788974,0.605590,0.481117,NaN,5.311050
2000-03-31,0.048741,0.049948,0.007877,-0.2,5.493947,1.590367,3.820005,0.773570,0.605590,-0.2,5.429869
2000-04-30,0.019855,0.020053,0.002494,-0.3,5.474707,1.605969,3.888418,0.482107,0.773570,0.7,5.493947
2000-05-31,0.031218,0.031710,0.003515,0.1,5.453072,1.633036,3.915735,0.355027,0.482107,-0.2,5.474707
...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,0.042240,0.043145,0.001956,0.8,4.082353,0.515687,2.761659,0.421353,0.389225,1.1,4.062892
2021-09-30,0.006814,0.006837,0.001526,0.7,4.078039,0.520634,2.767360,0.366907,0.421353,1.0,4.082353
2021-10-31,-0.005855,-0.005838,0.000690,1.5,4.068158,0.497318,2.750286,0.226861,0.366907,0.8,4.078039


样本外预测 Out-Of-Sample

In [16]:
reg_data[263:264]

,Log_return,Raw_return,MV,cpi,pd,pb,pe,to,lto,lcpi,lpd
month,,,,,,,,,,,
2021-12-31,0.021071,0.021294,0.00106,1.5,4.121076,0.538719,2.782958,0.340894,0.345915,1.5,4.091614


In [17]:
data = reg_data['2000-01':'2021-12'].copy()
model_pre = 0
mean_pre = 0

for i in range(int(len(data)/3), len(data) - 1):
    # 选择数据
    data_reg = data[0:i]
    model =smf.ols(formula='Raw_return ~ lcpi', data=data_reg).fit(displ=False)
    r_b = (model.predict(data[i:i+1]['lcpi']) - data[i:i+1]['Raw_return'])**2
    r_a = (np.mean(data_reg['Raw_return']) - data[i:i+1]['Raw_return'])**2
    r_b = r_b.values
    r_a = r_a.values
    model_pre = model_pre + r_b
    mean_pre = mean_pre + r_a

oos = 1 - model_pre/mean_pre
print("样本外R2是:",oos)

样本外R2是: [0.04234632]


In [18]:
data = reg_data['2000-01':'2021-12'].copy()

def out_of_sample(data,y,x,initial_sample_fractions):
    model_pre = 0
    mean_pre = 0

    initial_sample = int(len(data)*initial_sample_fractions)

    # 模型
    predictor_formula = "+".join(x)
    formula = f'{y} ~ {predictor_formula}'

    for i in range(initial_sample, len(data) - 1):
        # 选择数据
        data_reg = data[0:i]
        model =smf.ols(formula, data=data_reg).fit(displ=False)
        r_b = (model.predict(data[i:i+1][x]) - data[i:i+1]['Raw_return'])**2
        r_a = (np.mean(data_reg['Raw_return']) - data[i:i+1]['Raw_return'])**2
        r_b = r_b.values
        r_a = r_a.values
        model_pre = model_pre + r_b
        mean_pre = mean_pre + r_a

    oos = 1 - model_pre/mean_pre
    return oos
out_of_sample(data,'Raw_return',['lcpi'],0.9)
out_of_sample(data,'Raw_return',['lcpi','lpd'],0.9)
out_of_sample(data,'Raw_return',['lcpi','lpd','lto'],0.9)

array([-0.08252413])

array([-0.0649139])

array([-0.1080393])

In [19]:
sample_fractions = np.linspace(0.1,0.9,9)

results = {}

for i in sample_fractions:
    results[i] = out_of_sample(data,'Raw_return',['lcpi','lpd','lto'],i)

for i, j in results.items():
    print(i, j)

0.1 [0.02128731]
0.2 [0.0514447]
0.30000000000000004 [0.05293496]
0.4 [-0.04089868]
0.5 [-0.09731103]
0.6 [-0.1285119]
0.7000000000000001 [-0.33114049]
0.8 [-0.05637241]
0.9 [-0.1080393]


In [20]:
# Out of Sample Test
# benchmark model 是基准模型，为历史均值
# augment model 是比较模型，是我们要添加的x

# I want to write a function to calculate the out-of-sample R2


data = reg_data['2000-01':'2021-12'].copy()
benchmark = 0.00
augment = 0.00
ab = 0.00

for i in range(int(len(data) / 3), len(data) - 1):
    data_reg = data[0:i]
    model1 = smf.ols('Raw_return ~ lto', data_reg).fit(displ=False)
    r_a = (model1.predict(data[i:i + 1]['lto']) - data[i:i + 1]['Raw_return'])**2
    r_b = (np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])**2
    r_ab = (model1.predict(data[i:i + 1]['lto']) - data[i:i + 1]['Raw_return']) * (
        np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])
    r_a = r_a.values
    r_b = r_b.values
    r_ab = r_ab.values
    augment = augment + r_a
    benchmark += r_b
    ab += r_ab

oos = 1 - augment / benchmark
ENCNEW = ((benchmark - ab) / augment) * (len(data) - int(len(data) / 3))
MSEF = (benchmark - augment) / augment * (len(data) - int(len(data) / 3))

print('样本外的R方是', oos)
print('样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是', ENCNEW)
print('样本外的MSE-F是', MSEF)

样本外的R方是 [-0.03454616]
样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是 [2.76926819]
样本外的MSE-F是 [-5.87709315]


In [21]:
# benchmark model 是基准模型，为历史均值
# augment model 是比较模型，是我们要添加的x

data = reg_data['2000-01':'2021-12'].copy()
benchmark = 0.00
augment = 0.00
ab = 0.00

for i in range(int(len(data) / 3), len(data) - 1):
    data_reg = data[0:i]
    model1 = smf.ols('Raw_return ~ lto + lpd', data_reg).fit(displ=False)
    r_a = (model1.predict(data[i:i + 1][['lto','lpd']]) - data[i:i + 1]['Raw_return'])**2
    r_b = (np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])**2
    r_ab = (model1.predict(data[i:i + 1][['lto','lpd']]) - data[i:i + 1]['Raw_return']) * (
        np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])
    r_a = r_a.values
    r_b = r_b.values
    r_ab = r_ab.values
    augment = augment + r_a
    benchmark += r_b
    ab += r_ab

oos = 1 - augment / benchmark
ENCNEW = ((benchmark - ab) / augment) * (len(data) - int(len(data) / 3))
MSEF = (benchmark - augment) / augment * (len(data) - int(len(data) / 3))

print('样本外的R方是', oos)
print('样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是', ENCNEW)
print('样本外的MSE-F是', MSEF)

样本外的R方是 [-0.0211109]
样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是 [4.14118947]
样本外的MSE-F是 [-3.6387018]


In [22]:
data = reg_data['2000-01':'2021-12'].copy()
benchmark = 0.00
augment = 0.00
ab = 0.00

for i in range(int(len(data) / 3), len(data) - 1):
    data_reg = data[0:i]
    model1 = smf.ols('Raw_return ~ lto + lpd + lcpi', data_reg).fit(displ=False)
    r_a = (model1.predict(data[i:i + 1][['lto','lpd','lcpi']]) - data[i:i + 1]['Raw_return'])**2
    r_b = (np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])**2
    r_ab = (model1.predict(data[i:i + 1][['lto','lpd','lcpi']]) - data[i:i + 1]['Raw_return']) * (
        np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])
    r_a = r_a.values
    r_b = r_b.values
    r_ab = r_ab.values
    augment = augment + r_a
    benchmark += r_b
    ab += r_ab

oos = 1 - augment / benchmark
ENCNEW = ((benchmark - ab) / augment) * (len(data) - int(len(data) / 3))
MSEF = (benchmark - augment) / augment * (len(data) - int(len(data) / 3))

print('样本外的R方是', oos)
print('样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是', ENCNEW)
print('样本外的MSE-F是', MSEF)

样本外的R方是 [0.02007166]
样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是 [16.12179675]
样本外的MSE-F是 [3.60497068]


更具一般性的函数

In [23]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

def out_of_sample(data, response_var, predictor_vars):
    benchmark = 0.00
    augment = 0.00
    ab = 0.00

    # 构建模型公式
    predictor_formula = ' + '.join(predictor_vars)
    formula = f'{response_var} ~ {predictor_formula}'

    for i in range(int(len(data) / 3), len(data) - 1):
        data_reg = data.iloc[0:i]
        model = smf.ols(formula, data_reg).fit(displ=False)
        prediction = model.predict(data.iloc[i:i + 1][predictor_vars])
        actual = data.iloc[i:i + 1][response_var].values[0]

        r_a = (prediction - actual) ** 2
        r_b = (np.mean(data_reg[response_var]) - actual) ** 2
        r_ab = (prediction - actual) * (np.mean(data_reg[response_var]) - actual)

        augment += r_a.values
        benchmark += r_b
        ab += r_ab.values

    oos = 1 - augment / benchmark
    ENCNEW = ((benchmark - ab) / augment) * (len(data) - int(len(data) / 3))
    MSEF = (benchmark - augment) / augment * (len(data) - int(len(data) / 3))

    return oos, ENCNEW, MSEF


oos, ENCNEW, MSEF = out_of_sample(reg_data['2000-01':'2021-12'], 'Raw_return', ['lto','lpd'])
print('样本外的R方是', oos)
print('样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是', ENCNEW)
print('样本外的MSE-F是', MSEF)

样本外的R方是 [-0.0211109]
样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是 [4.14118947]
样本外的MSE-F是 [-3.6387018]


更具有一般性的函数

In [24]:
def out_of_sample(data, response_var, predictor_vars, initial_sample_fraction):
    benchmark = 0.00
    augment = 0.00
    ab = 0.00

    # 构建模型公式
    predictor_formula = ' + '.join(predictor_vars) # 将predictor_vars中的元素用+连接起来
    formula = f'{response_var} ~ {predictor_formula}' # 构建公式

    initial_sample_size = int(len(data) * initial_sample_fraction) # 计算初始样本的大小

    for i in range(initial_sample_size, len(data) - 1):
        data_reg = data.iloc[0:i] # 取出前i行的数据
        model = smf.ols(formula, data_reg).fit(displ=False) # 拟合模型
        prediction = model.predict(data.iloc[i:i + 1][predictor_vars]) # 预测值
        actual = data.iloc[i:i + 1][response_var].values[0] # 从data中取出第i行的response_var列的值

        r_a = (prediction - actual) ** 2 # 残差平方
        r_b = (np.mean(data_reg[response_var]) - actual) ** 2 # 均值平方
        r_ab = (prediction - actual) * (np.mean(data_reg[response_var]) - actual) # 残差乘均值

        augment += r_a.values   
        benchmark += r_b       
        ab += r_ab.values     

    oos = 1 - augment / benchmark
    ENCNEW = ((benchmark - ab) / augment) * (len(data) - initial_sample_size)
    MSEF = (benchmark - augment) / augment * (len(data) - initial_sample_size)

    return oos, ENCNEW, MSEF

# 使用函数
oos, ENCNEW, MSEF = out_of_sample(reg_data['2000-01':'2021-12'], 'Raw_return', ['lto','lpd'], initial_sample_fraction=3/4)
print('样本外的R方是', oos)
print('样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是', ENCNEW)
print('样本外的MSE-F是', MSEF)

样本外的R方是 [-0.04354756]
样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是 [-0.50062308]
样本外的MSE-F是 [-2.75420021]


字典

In [25]:
# 创建字典:
my_dict = {'name': 'Hang', 'age': 35}
my_dict

{'name': 'Hang', 'age': 35}

In [26]:
another_dict = dict(name="Bob", age=30)
another_dict

{'name': 'Bob', 'age': 30}

In [ ]:
# 访问元素:
# 使用键来访问对应的值。
print(my_dict['name']) 

In [27]:
# 修改元素:
# 可以直接通过键来修改对应的值。
my_dict['age'] = 26

In [28]:
# 添加新的键值对:
# 直接赋值一个新键及其值。
my_dict['address'] = 'Da Lian'

In [29]:
# 删除元素:
# 使用 del 语句或 pop 方法。
del my_dict['address']  # 删除键 'address'
age = my_dict.pop('age')  # 移除键 'age' 并返回其值

In [30]:
my_dict

{'name': 'Hang'}

In [31]:
# 检查键是否存在:
# 使用 in 关键字。
if 'name' in my_dict:
    print("Name is present in the dictionary")

Name is present in the dictionary


In [32]:
for key in my_dict:
    print(key, my_dict[key])


name Hang


In [33]:
for value in my_dict.values():
    print(value)

Hang


In [34]:
for key, value in my_dict.items():
    print(key, value)

name Hang


In [35]:
my_dict = {
    'fruits': ['apple', 'banana', 'cherry'],
    'numbers': [1, 2, 3],
    'colors': ['red', 'green', 'blue']
}

print(my_dict)

{'fruits': ['apple', 'banana', 'cherry'], 'numbers': [1, 2, 3], 'colors': ['red', 'green', 'blue']}


In [36]:
keys = my_dict.keys()  # 返回所有键
values = my_dict.values()  # 返回所有值
keys
values

dict_keys(['fruits', 'numbers', 'colors'])

dict_values([['apple', 'banana', 'cherry'], [1, 2, 3], ['red', 'green', 'blue']])

In [37]:
my_dict['numbers']

[1, 2, 3]

In [38]:
# 设置不同的初始样本比例
initial_sample_fractions = [0.3, 0.4, 0.5, 0.6, 0.7,0.8]

# 用于存储结果的字典
results = {}

for fraction in initial_sample_fractions:
    # 调用 out_of_sample 函数
    oos, ENCNEW, MSEF = out_of_sample(reg_data['2000-01':'2021-12'], 'Raw_return', ['lto','lpd'], initial_sample_fraction=fraction)
    
    # 将结果保存到字典
    results[fraction] = {
        'OOS': oos,
        'ENCNEW': ENCNEW,
        'MSEF': MSEF
    }

# 打印结果
for fraction, metrics in results.items():
    print(f"初始样本比例: {fraction}, 样本外的R方: {metrics['OOS']}, ENC-NEW: {metrics['ENCNEW']}, MSE-F: {metrics['MSEF']}")

初始样本比例: 0.3, 样本外的R方: [0.02035198], ENC-NEW: [8.95874762], MSE-F: [3.84333629]
初始样本比例: 0.4, 样本外的R方: [-0.04233821], ENC-NEW: [1.05501071], MSE-F: [-6.4583403]
初始样本比例: 0.5, 样本外的R方: [-0.09143245], ENC-NEW: [-2.02704548], MSE-F: [-11.05802095]
初始样本比例: 0.6, 样本外的R方: [-0.10868989], ENC-NEW: [-2.15524161], MSE-F: [-10.39165998]
初始样本比例: 0.7, 样本外的R方: [-0.2593602], ENC-NEW: [-5.90274597], MSE-F: [-16.47568047]
初始样本比例: 0.8, 样本外的R方: [-0.03087661], ENC-NEW: [-0.1194025], MSE-F: [-1.5874452]


In [39]:
results

{0.3: {'OOS': array([0.02035198]),
  'ENCNEW': array([8.95874762]),
  'MSEF': array([3.84333629])},
 0.4: {'OOS': array([-0.04233821]),
  'ENCNEW': array([1.05501071]),
  'MSEF': array([-6.4583403])},
 0.5: {'OOS': array([-0.09143245]),
  'ENCNEW': array([-2.02704548]),
  'MSEF': array([-11.05802095])},
 0.6: {'OOS': array([-0.10868989]),
  'ENCNEW': array([-2.15524161]),
  'MSEF': array([-10.39165998])},
 0.7: {'OOS': array([-0.2593602]),
  'ENCNEW': array([-5.90274597]),
  'MSEF': array([-16.47568047])},
 0.8: {'OOS': array([-0.03087661]),
  'ENCNEW': array([-0.1194025]),
  'MSEF': array([-1.5874452])}}

季度样本外

In [40]:
Qreg_data = reg_data.resample('Q').apply({
    'Raw_return':
    lambda x: np.exp(sum(np.log( 1 + x))) - 1,
    'to':
    lambda x: sum(x),
    'pd':
    lambda x: np.mean(x),
    'cpi':
    lambda x: sum(x),
})
Qreg_data['lto'] = Qreg_data['to'].shift(1)
Qreg_data['lpd'] = Qreg_data['pd'].shift(1)
Qreg_data['lcpi'] = Qreg_data['cpi'].shift(1)
Qreg_data

,Raw_return,to,pd,cpi,lto,lpd,lcpi
month,,,,,,,
2000-03-31,0.317318,1.860277,5.411622,0.3,NaN,NaN,NaN
2000-06-30,0.071041,1.310464,5.450551,0.3,1.860277,5.411622,0.3000
2000-09-30,-0.009310,1.021229,5.419918,0.8,1.310464,5.450551,0.3000
2000-12-31,0.085501,0.743491,5.593930,2.8,1.021229,5.419918,0.8000
2001-03-31,0.018954,0.544915,5.633003,2.0,0.743491,5.593930,2.8000
...,...,...,...,...,...,...,...
2020-12-31,0.079247,0.768780,4.086465,0.2,1.142121,4.058310,6.7715
2021-03-31,-0.008972,0.819679,4.128985,-0.1,0.768780,4.086465,0.2000
2021-06-30,0.043374,0.779270,4.143594,3.3,0.819679,4.128985,-0.1000


In [41]:
# benchmark model 是基准模型，为历史均值
# augment model 是比较模型，是我们要添加的x

data = Qreg_data['2000-01':'2021-12'].copy()
benchmark = 0.00
augment = 0.00
ab = 0.00

for i in range(int(len(data) / 3), len(data) - 1):
    data_reg = data[0:i]
    model1 = smf.ols('Raw_return ~ lto', data_reg).fit(displ=False)
    r_a = (model1.predict(data[i:i + 1]['lto']) - data[i:i + 1]['Raw_return'])**2
    r_b = (np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])**2
    r_ab = (model1.predict(data[i:i + 1]['lto']) - data[i:i + 1]['Raw_return']) * (
        np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])
    r_a = r_a.values
    r_b = r_b.values
    r_ab = r_ab.values
    augment = augment + r_a
    benchmark += r_b
    ab += r_ab

oos = 1 - augment / benchmark
ENCNEW = ((benchmark - ab) / augment) * (len(data) - int(len(data) / 3))
MSEF = (benchmark - augment) / augment * (len(data) - int(len(data) / 3))

print('样本外的R方是', oos)
print('样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是', ENCNEW)
print('样本外的MSE-F是', MSEF)

样本外的R方是 [0.05493354]
样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是 [6.44253614]
样本外的MSE-F是 [3.42947171]
